# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df= pd.read_csv('../output_data/weather_df.csv')
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.7297,-155.0900,69.53,94,90,8.05,US,1628082976
1,Port Alfred,-33.5906,26.8910,67.93,72,41,14.65,ZA,1628082977
2,Antalaha,-14.9003,50.2788,75.16,62,0,13.33,MG,1628083097
3,Olafsvik,64.8945,-23.7142,53.35,92,100,8.12,IS,1628082950
4,Punta Arenas,-53.1500,-70.9167,46.51,66,0,23.02,CL,1628083097
...,...,...,...,...,...,...,...,...,...
574,Vanimo,-2.6741,141.3028,79.07,83,100,3.00,PG,1628083279
575,Contamana,-7.3333,-75.0167,72.12,55,5,2.82,PE,1628083279
576,Balezino,57.9796,53.0106,78.04,67,100,9.13,RU,1628083279
577,Mazatán,14.8667,-92.4500,77.07,100,40,4.61,MX,1628083279


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations= weather_df[['Lat','Lng']]#.astype(float)

# Get the humidity rating
humidity= weather_df['Humidity']#.astype(float)

# Plot Heatmap, create a gmaps figure 
fig= gmaps.figure(center=(17.0, 4.0), zoom_level=2)

#create a heat layer (search the gmaps documentation - hint there's a heatmap_layer method)
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=4)

#add the layer to your figure
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = weather_df.loc[(weather_df['Max Temp'] >70) &
                          (weather_df['Max Temp'] <80) &
                          (weather_df['Wind Speed'] <10) &
                          (weather_df['Cloudiness'] ==0)].dropna()
                     

ideal_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,Vila Franca do Campo,37.7167,-25.4333,79.21,64,0,3.36,PT,1628083105
196,Hasaki,35.7333,140.8333,77.29,87,0,7.85,JP,1628083151
204,Sandusky,41.3501,-83.1332,71.80,76,0,1.23,US,1628083154
289,Caluquembe,-13.7833,14.6833,78.58,14,0,5.86,AO,1628083180
304,Christiana,-27.9140,25.1611,74.73,16,0,8.68,ZA,1628083185
318,Mount Darwin,-16.7725,31.5838,75.31,20,0,6.78,ZW,1628083189
338,Jiayuguan City,39.8167,98.3000,79.41,11,0,9.06,CN,1628083195
351,Enshi,30.3000,109.4833,79.61,76,0,0.47,CN,1628083199
371,Letlhakane,-21.4167,25.5833,77.47,19,0,3.62,BW,1628083211
442,Ladário,-19.0047,-57.6017,70.09,46,0,2.30,BR,1628083137


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df[["City","Country","Lat","Lng"]]
hotel_df['Hotel Name'] = ""
hotel_df.head(10)

<ipython-input-6-6a610bfcd4e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Hotel Name
37,Vila Franca do Campo,PT,37.7167,-25.4333,
196,Hasaki,JP,35.7333,140.8333,
204,Sandusky,US,41.3501,-83.1332,
289,Caluquembe,AO,-13.7833,14.6833,
304,Christiana,ZA,-27.9140,25.1611,
318,Mount Darwin,ZW,-16.7725,31.5838,
338,Jiayuguan City,CN,39.8167,98.3000,
351,Enshi,CN,30.3000,109.4833,
371,Letlhakane,BW,-21.4167,25.5833,
442,Ladário,BR,-19.0047,-57.6017,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()

    results= response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except:
        hotel_df.loc[index, "Hotel Name"] = "Hotel Not Found"
        
hotel_df

C:\Users\nick\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
37,Vila Franca do Campo,PT,37.7167,-25.4333,Vinha d'Areia Beach Hotel
196,Hasaki,JP,35.7333,140.8333,Villa Umi To Mori
204,Sandusky,US,41.3501,-83.1332,"Holiday Inn Express & Suites North Fremont, an..."
289,Caluquembe,AO,-13.7833,14.6833,Hotel Kalukembe
304,Christiana,ZA,-27.9140,25.1611,Vaal de Vue Guest House In Christiana
318,Mount Darwin,ZW,-16.7725,31.5838,Hotel Not Found
338,Jiayuguan City,CN,39.8167,98.3000,Jiayuguan Hotel
351,Enshi,CN,30.3000,109.4833,Moevenpick Enshi
371,Letlhakane,BW,-21.4167,25.5833,Hotel Phudzi
442,Ladário,BR,-19.0047,-57.6017,Nacional Palace Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))